In [1]:
import paddleocr

In [2]:
def read_text_from_image(image_path):
    paddle_ocr_en = paddleocr.PaddleOCR(lang='en', use_gpu=False, use_angle_cls=True, use_dilation=True, 
                                        max_text_length=50, label_list=['0', '45', '60', '90', '135', '180', '225', '270', '315'],
                                        enable_mkldnn=True, det_sast_polygon=True, gpu_mem=1250, use_mp=True)
    text_and_boxes = paddle_ocr_en.ocr(image_path, det=True, rec=True, cls=True)
    return " ".join([i[1][0] for i in text_and_boxes])

In [3]:
read_text_from_image("https://wowslider.com/images/data/images/slide1.png")

[2022/04/02 03:43:29] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=1250, image_dir=None, det_algorithm='DB', det_model_dir='C:\\Users\\sefa/.paddleocr/whl\\det\\en\\en_ppocr_mobile_v2.0_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=True, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=True, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='quad', det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, det_fce_box_type='poly', rec_algorithm='CRNN', rec_model_dir='C:\\Users\\sefa/.paddleocr/whl\\rec\\en\\en_number_mobile_v2.0_rec_infer', rec_image_shape='3, 32, 320', rec_batch_num=6, max_text_length=50, rec_c

100%|██████████| 13.9k/13.9k [00:00<00:00, 3.46MiB/s]

[2022/04/02 03:43:30] ppocr DEBUG: dt_boxes num : 3, elapse : 0.11096048355102539


[2022/04/02 03:43:30] ppocr DEBUG: cls num  : 3, elapse : 0.059000492095947266
[2022/04/02 03:43:31] ppocr DEBUG: rec_res num  : 3, elapse : 0.12999868392944336


'WOW Slider Awesome Slider for Non-Coders!'

In [4]:
read_text_from_image("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR5tWr7sIWSFkvAev8mYlUcD29Qhmf5MvRNTg&usqp=CAU")

[2022/04/02 03:43:31] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=1250, image_dir=None, det_algorithm='DB', det_model_dir='C:\\Users\\sefa/.paddleocr/whl\\det\\en\\en_ppocr_mobile_v2.0_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=True, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=True, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='quad', det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, det_fce_box_type='poly', rec_algorithm='CRNN', rec_model_dir='C:\\Users\\sefa/.paddleocr/whl\\rec\\en\\en_number_mobile_v2.0_rec_infer', rec_image_shape='3, 32, 320', rec_batch_num=6, max_text_length=50, rec_c

100%|██████████| 5.64k/5.64k [00:00<00:00, 2.82MiB/s]

[2022/04/02 03:43:32] ppocr DEBUG: dt_boxes num : 1, elapse : 0.0399632453918457
[2022/04/02 03:43:32] ppocr DEBUG: cls num  : 1, elapse : 0.04400229454040527
[2022/04/02 03:43:32] ppocr DEBUG: rec_res num  : 1, elapse : 0.037999868392944336


'TEXT'

In [5]:
from flask import Flask
from flask import jsonify, request
from text_extraction import download_image, read_text_from_image
import filetype
import os
import logging

app = Flask(__name__)

LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.INFO)
logging.basicConfig()


@app.route('/text_recognition/<path:image_url>')
def text_recognition(image_url):
    image_path = download_image(image_url)
    if image_path and filetype.is_image(image_path):
        free_text = read_text_from_image(image_path)
        LOGGER.info(f"URL LINK : {image_url}")
        LOGGER.info(f"EXTRACTED TEXT : {free_text}")
        print("-" * 120)
        os.remove(image_path)
        return jsonify(data={'extracted_text': free_text})
    else:
        os.remove(image_path)
        return jsonify(data={'extracted_text': f'{image_url} not found or it is not an image file'}), 500


if __name__ == '__main__':
    app.run(port=8080)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)


[2022/04/02 03:43:55] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=1250, image_dir=None, det_algorithm='DB', det_model_dir='C:\\Users\\sefa/.paddleocr/whl\\det\\en\\en_ppocr_mobile_v2.0_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=True, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=True, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='quad', det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, det_fce_box_type='poly', rec_algorithm='CRNN', rec_model_dir='C:\\Users\\sefa/.paddleocr/whl\\rec\\en\\en_number_mobile_v2.0_rec_infer', rec_image_shape='3, 32, 320', rec_batch_num=6, max_text_length=50, rec_c

INFO:__main__:URL LINK : https://s3-us-west-2.amazonaws.com/courses-images/wp-content/uploads/sites/1844/2017/05/04193300/alignright.png
INFO:__main__:EXTRACTED TEXT : Buy 1 Get 1 Free Notebooks and planners While supplies last Sale ends September 25
INFO:werkzeug:127.0.0.1 - - [02/Apr/2022 03:43:56] "GET /text_recognition/https://s3-us-west-2.amazonaws.com/courses-images/wp-content/uploads/sites/1844/2017/05/04193300/alignright.png HTTP/1.1" 200 -


------------------------------------------------------------------------------------------------------------------------
[2022/04/02 03:44:22] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=1250, image_dir=None, det_algorithm='DB', det_model_dir='C:\\Users\\sefa/.paddleocr/whl\\det\\en\\en_ppocr_mobile_v2.0_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=True, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=True, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='quad', det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, det_fce_box_type='poly', rec_algorithm='CRNN', rec_model_dir='C:\\Users\\sefa/.paddleocr

INFO:__main__:URL LINK : https://watermarkly.com/images/add-text-to-photo/sample3.webp
INFO:__main__:EXTRACTED TEXT : If you want to go fast, go alone. If you want to go far, gotogether.
INFO:werkzeug:127.0.0.1 - - [02/Apr/2022 03:44:23] "GET /text_recognition/https://watermarkly.com/images/add-text-to-photo/sample3.webp HTTP/1.1" 200 -


------------------------------------------------------------------------------------------------------------------------
